In [1]:
import nbimporter
from librerias_funciones import *

In [2]:
# Función para manejar la selección
def handle_variable_selection(change):
    global intervalo_temporal_busqueda
    intervalo_temporal_busqueda = change['new']

In [3]:
url = 'https://ticjob.es/esp/busqueda'
portal_busqueda = 'ticjob'

usuario = 'bruno.octavio.tomas@gmail.com'
password = 'password123'
trabajor_a_buscar = 'Data Scientist'
ubicacion_a_buscar = 'España'
nombres_columnas = ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", 
                    "experiencia", "tipo_contrato", "salario", "fecha_scrapeo","url", "portal"]

intervalo_temporal_busqueda = 2

In [4]:
intervalo_temporal_busqueda = timedelta(days = intervalo_temporal_busqueda)

In [5]:
driver = webdriver.Chrome()
driver.get(url)
sleep(1)
driver.maximize_window()

In [6]:
# Seleccionamos el puesto a buscar
#driver.find_element(By.ID, 'keywords-input').send_keys(trabajor_a_buscar)

In [7]:
#seleccionamos la ubicacion a buscar

#driver.find_element(By.ID, 'location-map-dd').click()

In [8]:
# Ordenamos los resultados por fecha

driver.find_element(By.CLASS_NAME, 'sort-by-date-container').click()

sleep(5)

In [ ]:
fecha_scrapeo = datetime.now().date()

while True:
    data = []
    ofertas = driver.find_elements(By.CLASS_NAME, 'job-card')

    for oferta in ofertas:
        #Extraemos los datos con selenium, ya que al obtener la url de la pagina nos da los de una busqueda generica sin nuetros criterios
        #Extraemos solo los datos del intervalo temporal seleccionado
        fecha = oferta.find_element(By.CSS_SELECTOR, 'div[class = "job-card-label date-field"]').text
        fecha = datetime.strptime(fecha, '%d/%m/%Y').date()

        if fecha < fecha_scrapeo - intervalo_temporal_busqueda:
            break

        url = oferta.find_element(By.TAG_NAME, 'a').get_attribute('href')

        response = requests.get(url)
        
        #Extraemos los datos con soup
        soup = BeautifulSoup(response.text, "html.parser")
        
        titulo = extraccion_datos_try_except(soup, 'h1', {'id' : 'job-title'})
        
        empresa = soup.find('a', class_ = 'company-image')['title']
        
        descripcion = extraccion_datos_try_except(soup, 'div', {'class' : 'job-offer job-offer-content'})
        
        ubicacion = extraccion_datos_try_except(soup, 'li', {'class' : 'multi-job-location-apply'})
        if (ubicacion != np.nan) and (';' in ubicacion):
            ubicacion = ubicacion.split(';')
        
        experiencia = extraccion_datos_try_except(soup, 'li', {'id' : 'summaryExp'})

        localizacion = soup.find('li', class_ = 'multi-job-location-apply')
        tipo_contrato = localizacion.find_next('li').get_text(strip = True)

        salario = extraccion_datos_try_except(soup, 'li', {'id' : 'summarySalary'})
        # Eliminamos los salarios == '0' ya que no son datos reales que se muestren en la pagina, ya que corresponden con campos vacios
        salario = salario if salario != '0' else np.nan
        
        herramientas = soup.find('div', class_ = 'search-criteria-tags').find_all('a')
        herramientas = [herramienta.text for herramienta in herramientas]
        
        data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, experiencia, tipo_contrato, salario, fecha_scrapeo, url, portal_busqueda])
        sleep(1)
        
    
    #Guardamos cada vez que termina de sacrapear una pagina
    guardar_datos(portal_busqueda = portal_busqueda, nombres_columnas = nombres_columnas, data = data)
    
    paginas_totales = driver.find_element(By.CLASS_NAME, 'page-list').text.split('\n')[-1]
    paginas_totales = int(paginas_totales)
    pagina_actual = int(driver.find_element(By.CLASS_NAME, 'current').text)

    # Pasamos a la siguiente pagina hasta llegar a la ultima
    if paginas_totales != pagina_actual:
        # Pasamos a la siguiente pagina
        siguiente_pagina = driver.find_element(By.CLASS_NAME, 'next')
        driver.execute_script("arguments[0].scrollIntoView();", siguiente_pagina)
        siguiente_pagina.click()
        sleep(2)
    else:
        driver.quit()
        print('Scrapeo completado')
        break

In [11]:
# Actualizamos el fichero de control de extraccion de datos
actualizar_fichero_control_extracciones_datos(portal_busqueda, fecha_scrapeo)